In [2]:
import requests
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import MetaTrader5 as mt5
mt5.initialize()
# Replace following with your MT5 Account Login
account=51434456 # 
password="9UpBvVzc"
server = 'ICMarkets-Demo'

In [ ]:
BRITISH POUND - CHICAGO MERCANTILE EXCHANGE                                                             Code-096742
Commitments of Traders - Futures Only, November 28, 2023                                                           
-------------------------------------------------------------------------------------------------------------------
     :   Total  :                        Reportable Positions                                :   Nonreportable
     :----------------------------------------------------------------------------------------     Positions
     :   Open   :           Non-Commercial       :     Commercial      :       Total         :
     : Interest :   Long   :  Short   : Spreading:   Long   :  Short   :   Long   :  Short   :   Long   :  Short
-------------------------------------------------------------------------------------------------------------------
     :          : (CONTRACTS OF GBP 62,500)                                                  :
     :          :                                                                            :
All  :   217,372:    61,296     69,191     11,756    118,253    108,189    191,305    189,136:    26,067     28,236
Old  :   217,372:    61,296     69,191     11,756    118,253    108,189    191,305    189,136:    26,067     28,236
Other:         0:         0          0          0          0          0          0          0:         0          0
     :          :                                                                            :
     :          :          Changes in Commitments from: November 21, 2023                    :
     :     8,715:    17,996       -207      2,442    -11,153      9,991      9,285     12,226:      -570     -3,511
     :          :                                                                            :
     :          :    Percent of Open Interest Represented by Each Category of Trader         :
All  :     100.0:      28.2       31.8        5.4       54.4       49.8       88.0       87.0:      12.0       13.0
Old  :     100.0:      28.2       31.8        5.4       54.4       49.8       88.0       87.0:      12.0       13.0
Other:     100.0:       0.0        0.0        0.0        0.0        0.0        0.0        0.0:       0.0        0.0
     :          :                                                                            :
     :# Traders :              Number of Traders in Each Category                            :
All  :       120:        22         30         16         32         50         63         88:
Old  :       120:        22         30         16         32         50         63         88:
Other:         0:         0          0          0          0          0          0          0:
     :----------------------------------------------------------------------------------------------------
     :             Percent of Open Interest Held by the Indicated Number of the Largest Traders
     :                          By Gross Position                       By Net Position
     :               4 or Less Traders     8 or Less Traders     4 or Less Traders     8 or Less Traders
     :                 Long:     Short       Long      Short:      Long      Short       Long      Short
     :----------------------------------------------------------------------------------------------------
All  :                 36.5       21.4       54.1       34.0       33.7       21.4       49.7       34.0
Old  :                 36.5       21.4       54.1       34.0       33.7       21.4       49.7       34.0
Other:                  0.0        0.0        0.0        0.0        0.0        0.0        0.0        0.0
 
JAPANESE YEN - CHICAGO MERCANTILE EXCHANGE                                                              Code-097741
Commitments of Traders - Futures Only, November 28, 2023                                                           
-------------------------------------------------------------------------------------------------------------------
     :   Total  :                        Reportable Positions                                :   Nonreportable
     :----------------------------------------------------------------------------------------     Positions
     :   Open   :           Non-Commercial       :     Commercial      :       Total         :
     : Interest :   Long   :  Short   : Spreading:   Long   :  Short   :   Long   :  Short   :   Long   :  Short
-------------------------------------------------------------------------------------------------------------------
     :          : (CONTRACTS OF JPY 12,500,000)                                              :
     :          :                                                                            :
All  :   260,781:    30,461    139,698      5,804    180,134     76,442    216,399    221,944:    44,382     38,837
Old  :   260,781:    30,461    139,698      5,804    180,134     76,442    216,399    221,944:    44,382     38,837
Other:         0:         0          0          0          0          0          0          0:         0          0
     :          :                                                                            :
     :          :          Changes in Commitments from: November 21, 2023                    :
     :    -4,122:    -8,775     -4,992       -459      1,302      3,038     -7,932     -2,413:     3,810     -1,709
     :          :                                                                            :
     :          :    Percent of Open Interest Represented by Each Category of Trader         :
All  :     100.0:      11.7       53.6        2.2       69.1       29.3       83.0       85.1:      17.0       14.9
Old  :     100.0:      11.7       53.6        2.2       69.1       29.3       83.0       85.1:      17.0       14.9
Other:     100.0:       0.0        0.0        0.0        0.0        0.0        0.0        0.0:       0.0        0.0
     :          :                                                                            :
     :# Traders :              Number of Traders in Each Category                            :
All  :       153:        31         45         14         53         44         95         92:
Old  :       153:        31         45         14         53         44         95         92:
Other:         0:         0          0          0          0          0          0          0:
     :----------------------------------------------------------------------------------------------------
     :             Percent of Open Interest Held by the Indicated Number of the Largest Traders
     :                          By Gross Position                       By Net Position
     :               4 or Less Traders     8 or Less Traders     4 or Less Traders     8 or Less Traders
     :                 Long:     Short       Long      Short:      Long      Short       Long      Short
     :----------------------------------------------------------------------------------------------------
All  :                 32.2       25.1       47.1       39.4       32.2       25.1       46.4       39.4
Old  :                 32.2       25.1       47.1       39.4       32.2       25.1       46.4       39.4
Other:                  0.0        0.0        0.0        0.0        0.0        0.0        0.0        0.0
 
EURO FX - CHICAGO MERCANTILE EXCHANGE                                                                   Code-099741
Commitments of Traders - Futures Only, November 28, 2023                                                           
-------------------------------------------------------------------------------------------------------------------
     :   Total  :                        Reportable Positions                                :   Nonreportable
     :----------------------------------------------------------------------------------------     Positions
     :   Open   :           Non-Commercial       :     Commercial      :       Total         :
     : Interest :   Long   :  Short   : Spreading:   Long   :  Short   :   Long   :  Short   :   Long   :  Short
-------------------------------------------------------------------------------------------------------------------
     :          : (CONTRACTS OF EUR 125,000)                                                 :
     :          :                                                                            :
All  :   727,138:   233,454     90,289     22,287    388,784    565,099    644,525    677,675:    82,613     49,463
Old  :   727,138:   233,454     90,289     22,287    388,784    565,099    644,525    677,675:    82,613     49,463
Other:         0:         0          0          0          0          0          0          0:         0          0
     :          :                                                                            :
     :          :          Changes in Commitments from: November 21, 2023                    :
     :    14,774:     2,359    -11,152      5,323      7,969     18,466     15,651     12,637:      -877      2,137
     :          :                                                                            :
     :          :    Percent of Open Interest Represented by Each Category of Trader         :
All  :     100.0:      32.1       12.4        3.1       53.5       77.7       88.6       93.2:      11.4        6.8
Old  :     100.0:      32.1       12.4        3.1       53.5       77.7       88.6       93.2:      11.4        6.8
Other:     100.0:       0.0        0.0        0.0        0.0        0.0        0.0        0.0:       0.0        0.0
     :          :                                                                            :
     :# Traders :              Number of Traders in Each Category                            :
All  :       301:        88         48         37        124         89        226        160:
Old  :       301:        88         48         37        124         89        226        160:
Other:         0:         0          0          0          0          0          0          0:
     :----------------------------------------------------------------------------------------------------
     :             Percent of Open Interest Held by the Indicated Number of the Largest Traders
     :                          By Gross Position                       By Net Position
     :               4 or Less Traders     8 or Less Traders     4 or Less Traders     8 or Less Traders
     :                 Long:     Short       Long      Short:      Long      Short       Long      Short
     :----------------------------------------------------------------------------------------------------
All  :                 18.8       48.6       28.8       66.2       18.2       48.3       27.6       65.6
Old  :                 18.8       48.6       28.8       66.2       18.2       48.3       27.6       65.6
Other:                  0.0        0.0        0.0        0.0        0.0        0.0        0.0        0.0

In [5]:
import requests
import re

# URL of the COT report
url = 'https://www.cftc.gov/dea/futures/deacmelf.htm'

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Get the text content of the report
    report_text = response.text

    # Define a regex pattern to match the data rows
    
    # This pattern might need to be adjusted depending on the specific structure of the report
    pattern = r'(?P<instrument>.+?):\s+(?P<open_interest>\d+).*?\n.*?\n.*?\n.*?\nAll\s+:\s+(?P<all_data>.+?)\n'
    matches = re.finditer(pattern, report_text, re.DOTALL)

    # Iterate over matches and extract data
    for match in matches:
        instrument = match.group('instrument').strip()
        open_interest = match.group('open_interest').strip()
        all_data = match.group('all_data').strip().split()

        # Process 'all_data' to extract specific fields as needed
        # Example: extracting the first few fields
        non_commercial_long = all_data[1]
        non_commercial_short = all_data[2]

        print(f"Instrument: {instrument}, Open Interest: {open_interest}, Non-Commercial Long: {non_commercial_long}, Non-Commercial Short: {non_commercial_short}")

else:
    print(f"Failed to retrieve data: {response.status_code}")

Instrument: <!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html lang="en-US" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">
<head>
<title>CFTC Commitments of Traders Long Report - CME (Futures Only)</title>
<!--begin meta-->
<meta http-equiv="Content-Type" content="text/html; charset=iso-8859-1" />
<meta http-equiv="Pragma" content="no-cache">
<meta http-equiv="Expires" content="0">
<meta name="description" content="This is the viewable version of the most recent release of the CME long form futures only commitments report." />
<meta name="file" content="deacmelf.htm" />
<meta name="keywords" content="commitments, long, CFTC, CME, Futures only" />
<meta name="office" content="dea" />
<!--end meta-->

<script type="text/javascript" src="http://www.google-analytics.com/ga.js"></script><script type='text/javascript'>var pageTracker = _gat._getTracker("UA-21047137-1");	pageTracker._trackPageview();</script><sc